# Inference Notebook Template

Country Level 1 (L1) means-

* Without Continent as cue
* Without Explicit Country list


In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True"

🔨 TO BE MODIFIED 🔨

In [ ]:
# TO BE MODIFIED

!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ax2cnbu0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-ax2cnbu0
  Resolved https://github.com/huggingface/transformers.git to commit 76fc50a1527a7db593a6057903b749598f7000a9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.0/502.0 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=11349228 sha256=988914959ace55258a162b0f5930bc0c7972b3d59a89bc302f7f04cf23cef14e
  Stored in directory: /tmp/pip-ephem-wheel-cache-encfwnys/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Foun

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## 📚 Helper: Save any results dict to JSON

In [ ]:
import json, os


def save_results(data: dict,
                 model_name: str,
                 variant: str,
                 task: str,
                 task_level: str,
                 prompt_level: str,
                 run_count: str,
                 output_dir: str = "/content/drive/MyDrive/RQ1_Experiments/results"):
    model_dir = os.path.join(output_dir, model_name)
    os.makedirs(model_dir, exist_ok=True)
    fname    = f"RQ1_{model_name}_{variant}_{task}_{task_level}_{prompt_level}_{run_count}.json"
    out_path = os.path.join(model_dir, fname)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    print(f" Saved results to {out_path}")


🔨 TO BE MODIFIED 🔨

### **Config**

In [ ]:
MODEL_NAME   = 'InternVL3-8B'
VARIANT      = '8B'
RUN_COUNT    = 'r1'
PROMPT_LEVEL = 'prompt1'


folder_path  = "/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1"

TASK = 'country'

# output root in Drive
OUTPUT_DIR = "/content/drive/MyDrive/RQ1_Experiments/results"

# Generation settings
GEN_TEMPERATURE     = 0.3
GEN_MAX_NEW_TOKENS  = 1024
GEN_DO_SAMPLE       = False

# InternVL3 vision tiling
IMAGE_SIZE     = 448
MAX_NUM_TILES  = 4
FALLBACK_TILES = 3

# Quantization: use "none" first. If OOM on load, set to "bnb-8bit".
QUANTIZATION = "none"   # "none" | "bnb-8bit"


### **Dataset discovery**

In [ ]:
import os

def get_all_file_paths(root_dir):
    file_paths = []
    stack = [root_dir]
    while stack:
        current_dir = stack.pop()
        with os.scandir(current_dir) as it:
            for entry in it:
                if entry.is_file() and entry.name.lower().endswith((".jpg",".jpeg",".png",".webp",".bmp")):
                    file_paths.append(entry.path)
                elif entry.is_dir():
                    stack.append(entry.path)
    return sorted(file_paths)

assert os.path.exists(folder_path), f"folder_path does not exist: {folder_path}"
image_paths = get_all_file_paths(folder_path)
print(f"Found {len(image_paths)} files.")


Found 360 files.


In [ ]:
print(image_paths)

['/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B1_MIDDLE.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B3_MIDDLE.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B4_MIDDLE.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B7_MIDDLE.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B8_MIDDLE.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B9_MIDDLE.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Cape Verde/Africa_Cape Verde_CV1_LAST.PNG', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Cape Verde/Africa_Cape Verde_CV2_LAST.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Cape Verde/Africa_Cape Verde_CV3_LAST.JPG', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Cape Verde/Africa_Cape Verd

# Variant: InternVL3 8B

https://huggingface.co/OpenGVLab/InternVL3-8B

## 1️⃣ Prompt

🔨 TO BE MODIFIED 🔨

In [ ]:
# TO BE MODIFIED

## Prompt (Country L1)

PROMPTS = {
    "prompt1": """Analyze the image. Answer two things:

1) Which country is this most likely from?

2) What is the most suitable label: political expression, vandalism, decoration, cultural identity, or advertisement?""",

    "prompt2": """You are a cultural anthropologist viewing street visuals. For the mural in image, give these four pieces of info:

- COUNTRY
- LABEL (choose exactly one: political expression / vandalism / decoration / cultural identity / advertisement)
- CONFIDENCE (0–100%)
- TOP CUE (2–3 sentences describing the key visual/cultural cue)

Be concise and calibrate CONFIDENCE to how strongly the visual cues support your claims.""",

    "prompt3": """Moderate a brief debate among a Geographer, a Street Artist, and a Museum Curator about this image. For each expert, give:

- ROLE: <role name>
- COUNTRY
- LABEL (political expression / vandalism / decoration / cultural identity / advertisement)
- SHORT_RATIONALE (one concise sentence)

After the three positions, write a 2-paragraph moderator resolution:
1) Compare the three rationales and state which COUNTRY and LABEL you choose and why (trade-offs).
2) List 2 visual cues that were decisive and mention one alternative explanation you rejected.

Finish with: COUNTRY and LABEL (each on its own line).""",

    "prompt4": """You are a cultural anthropologist & artist. For the mural in image, provide:

- COUNTRY: two most likely countries with percentages (sum ≈100), e.g. "Iceland: 70%, Benin: 30%"
- WHY COUNTRY: 2–3 sentences explaining how visual cues suggest the chosen countries .

- LABEL: choose from (political expression, vandalism, decoration, cultural identity, advertisement) and also assign probabilities for all five labels (sum ≈100)
- WHY LABEL: 2–3 sentences linking cues to chosen label.

- TOP CUES: list three concrete cues that were decisive (1–6 words each)
- BIAS NOTE: 2–3 sentences reflecting how cultural bias might have affected the probability estimates.

Constraints:
- Avoid equal continent splits (e.g. 50/50). Show one as more likely.
- Keep WHY COUNTRY and WHY LABEL tied to visible evidence (colors, symbols, technique, location clues etc)."""
}

PROMPT_ORDER = ["prompt1","prompt2","prompt3","prompt4"]



## 2️⃣ Load Processor and Model

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

🔨 TO BE MODIFIED 🔨

In [ ]:
# TO BE MODIFIED

from transformers import AutoModel, AutoTokenizer, AutoProcessor
import torch, os
from PIL import Image
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

repo_id = "OpenGVLab/InternVL3-8B"

model = AutoModel.from_pretrained(
    repo_id,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True
).eval().cuda()

processor = AutoProcessor.from_pretrained(
    repo_id,
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(repo_id, trust_remote_code=True, use_fast=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

configuration_internvl_chat.py: 0.00B [00:00, ?B/s]

configuration_intern_vit.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL3-8B:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL3-8B:
- configuration_internvl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`torch_dtype` is deprecated! Use `dtype` instead!
Unrecognized keys in `rope_parameters` for 'rope_type'='dynamic': {'rope_theta'}


...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

## 3️⃣ Inference

In [ ]:
import tempfile

🔨 TO BE MODIFIED 🔨

In [ ]:
# TO BE MODIFIED

import numpy as np

def infer_img(image_path: str, prompt: str):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")

    # Build question with the correct image token
    question = prompt if "<image>" in prompt else "<image>\n" + prompt

    # Read & minimally preprocess to a single 448x448 tile
    img = Image.open(image_path).convert("RGB").resize((448, 448))
    arr = np.array(img, dtype=np.float32) / 255.0
    # ImageNet normalize inline
    arr[..., 0] = (arr[..., 0] - 0.485) / 0.229
    arr[..., 1] = (arr[..., 1] - 0.456) / 0.224
    arr[..., 2] = (arr[..., 2] - 0.406) / 0.225
    # HWC -> BCHW
    pixel_values = torch.from_numpy(arr).permute(2, 0, 1).unsqueeze(0)
    pixel_values = pixel_values.to(torch.bfloat16)
    if any(p.is_cuda for p in model.parameters()):
        pixel_values = pixel_values.cuda()

    # Generation config from your globals
    gen_cfg = dict(
        max_new_tokens=GEN_MAX_NEW_TOKENS,
        do_sample=GEN_DO_SAMPLE,
        temperature=GEN_TEMPERATURE
    )

    # InternVL3 chat API
    with torch.inference_mode():
        response = model.chat(
            tokenizer,
            pixel_values,
            question,
            gen_cfg,
            history=None,
            return_history=False
        )

    return response.strip()



## 4️⃣ Package & Save to JSON

In [ ]:
## Package & Save to JSON

import re, json, torch, tempfile, os
from PIL import Image

def strip_code_fence(s: str) -> str:
    if not isinstance(s, str):
        return s
    s = s.strip()
    s = re.sub(r"^``` ?json\s*", "", s, flags=re.I)
    s = re.sub(r"```$", "", s)
    return s.strip()

def try_parse_json_from_string(s: str):
    cleaned = strip_code_fence(s)
    try:
        return json.loads(cleaned), None
    except json.JSONDecodeError:
        m = re.search(r"(\{(?:.|\s)*\}|\[(?:.|\s)*\])", cleaned)
        if m:
            try:
                return json.loads(m.group(1)), None
            except json.JSONDecodeError as e:
                return None, f"JSON decode failed for extracted substring: {e}"
        return None, "no JSON found or JSON invalid"

def normalize(parsed):
    if isinstance(parsed, list):
        if len(parsed) == 1 and isinstance(parsed[0], dict):
            return parsed[0]
        return {"json_list": parsed}
    return parsed

_LABELS = [
    "political expression","vandalism","decoration","cultural identity","advertisement"
]

def _derive_country(text: str):
    m = re.search(r"\bfrom\s+([A-Z][A-Za-z .'-]{2,})", text)
    if m:
        return m.group(1).strip().rstrip(". ,;:")
    # fallbacks
    m2 = re.search(r"\bCOUNTRY\s*[:\-]\s*([A-Z][A-Za-z .'-]{2,})", text, flags=re.I)
    if m2:
        return m2.group(1).strip().rstrip(". ,;:")
    return None

def _derive_label(text: str):
    t = text.lower()
    for lab in _LABELS:
        if re.search(rf"\b{re.escape(lab.lower())}\b", t):
            return lab, "exact"
    if "political" in t:
        return "political expression", "synonym"
    if "identity" in t:
        return "cultural identity", "synonym"
    if re.search(r"\bads?\b|\badvert\b|\badvertising\b", t):
        return "advertisement", "synonym"
    return None, None

def _parse_confidence(text: str):
    m = re.search(r"(?:confidence[^0-9]{0,12})(\d{1,3})\s*%", text, flags=re.I)
    if m:
        v = int(m.group(1))
        return max(0, min(100, v))
    m2 = re.search(r"\b(\d{1,3})\s*%", text)
    if m2:
        v = int(m2.group(1))
        if 0 <= v <= 100:
            return v
    return None

def _parse_prob_table(text: str, keys):
    found = {}
    for line in text.splitlines():
        m = re.search(r"^\s*([A-Za-z][A-Za-z\s\-\(\)]+)\s*:\s*(\d{1,3})\s*%\s*$", line)
        if not m:
            continue
        k_raw, v = m.group(1).strip(), int(m.group(2))
        # match against known keys
        k_norm = None
        for k in keys:
            if k_raw.lower() == k.lower():
                k_norm = k
                break
        # label synonyms
        if not k_norm:
            if k_raw.lower() in {"political","political expr","political-expression"}:
                k_norm = "political expression"
            elif k_raw.lower() in {"cultural","identity","cultural-identity"}:
                k_norm = "cultural identity"
            elif k_raw.lower() in {"ads","ad","advert"}:
                k_norm = "advertisement"
        if k_norm:
            found[k_norm] = max(0, min(100, v))
    return found or None

# keep one results array per prompt
results_per_prompt = {pl: [] for pl in PROMPT_ORDER}

TASK = 'country'
TASK_LEVEL = 'L1'  # Country Level 1

for i, image_path in enumerate(image_paths, start=1):
    print(f"\nProcessing {i}/{len(image_paths)}: {image_path}")

    parts = image_path.split(os.sep)
    dataset_continents = {"Africa","Asia","Europe","North America","South America","Oceania","Antarctica"}
    dataset_continent = next((p for p in parts if p in dataset_continents), None)
    dataset_country = None
    if dataset_continent:
        try:
            idx = parts.index(dataset_continent)
            dataset_country = parts[idx+1] if idx+1 < len(parts) else None
        except ValueError:
            pass
    dataset_split_hint = next((p for p in parts if p.upper() in {"TOP","MIDDLE","LAST"}), None)

    # iterate 4 prompts
    for pl in PROMPT_ORDER:
        PROMPT_LEVEL = pl
        prompt_text = PROMPTS[pl]

        try:
            raw_output = infer_img(image_path, prompt_text)
            print(f"  ✔ {pl}: infer_img() full-res ok")
        except RuntimeError as e:
            msg = str(e).lower()
            if "cuda out of memory" in msg:
                torch.cuda.empty_cache()
                print(f"  ⚠ {pl}: OOM on full-res. Resizing to 448 and retrying…")
                try:
                    img = Image.open(image_path).convert("RGB").resize((448, 448))
                    with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as tmp:
                        tmp_path = tmp.name
                        img.save(tmp_path, format="JPEG")
                    raw_output = infer_img(tmp_path, prompt_text)
                    print(f"  ✔ {pl}: infer_img() resized ok")
                except Exception as e2:
                    print(f"  ❌ {pl}: failed after resize: {e2}")
                    raw_output = None
            else:
                print(f"  ❌ {pl}: runtime error: {e}")
                raw_output = None
        except Exception as e:
            print(f"  ❌ {pl}: unexpected error: {e}")
            raw_output = None

        # normalize output (prefer JSON if present)
        parse_notes = []
        if raw_output is None:
            output_value = None
            parse_notes.append("no_output")
        elif isinstance(raw_output, (dict, list)):
            output_value = normalize(raw_output)
        else:
            parsed, perr = try_parse_json_from_string(raw_output)
            if parsed is not None:
                output_value = normalize(parsed)
                parse_notes.append("json_ok")
            else:
                output_value = raw_output
                parse_notes.append(f"json_failed:{perr}" if perr else "json_failed")

        # post-hoc extraction for analysis
        text_for_parse = (
            output_value if isinstance(output_value, str)
            else json.dumps(output_value, ensure_ascii=False) if output_value is not None else ""
        )
        country_guess = _derive_country(text_for_parse)
        label_guess, label_source = _derive_label(text_for_parse)
        confidence = _parse_confidence(text_for_parse)
        country_probs = _parse_prob_table(text_for_parse, keys=[])
        label_probs   = _parse_prob_table(text_for_parse, keys=_LABELS)

        rec = {
            "image_path":         image_path,
            "model":              MODEL_NAME,
            "variant":            VARIANT,
            "task":               TASK,
            "task_level":         TASK_LEVEL,
            "prompt_set":         "RQ1",
            "prompt_level":       PROMPT_LEVEL,
            "run_count":          RUN_COUNT,
            "prompt":             prompt_text,
            "output":             output_value,

            # path-derived hints
            "dataset_continent":  dataset_continent,
            "dataset_country":    dataset_country,
            "dataset_split_hint": dataset_split_hint,

            # reproducibility
            "temperature":        GEN_TEMPERATURE,
            "max_new_tokens":     GEN_MAX_NEW_TOKENS,
            "do_sample":          GEN_DO_SAMPLE,
            "model_repo":         repo_id,
            "device":             ("cuda" if any(p.is_cuda for p in model.parameters()) else "cpu"),
            "dtype":              str(next(model.parameters()).dtype).replace("torch.", ""),

            # analysis helpers
            "country_guess":      country_guess,
            "label_guess":        label_guess,
            "label_guess_source": label_source,
            "confidence":         confidence,
            "country_probs":      country_probs,
            "label_probs":        label_probs,
            "parse_notes":        parse_notes
        }
        results_per_prompt[pl].append(rec)

        # save per-prompt continuously
        save_results(
            results_per_prompt[pl],
            MODEL_NAME, VARIANT, TASK, TASK_LEVEL,
            pl, RUN_COUNT
        )

    torch.cuda.empty_cache()
    print("------------------------------------------------------")

print("✅ Done.")



Processing 1/360: /content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B1_MIDDLE.png


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


  ✔ prompt1: infer_img() full-res ok
 Saved results to r1/InternVL3-8B/RQ1_InternVL3-8B_8B_country_L1_prompt1.json


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


  ✔ prompt2: infer_img() full-res ok
 Saved results to r1/InternVL3-8B/RQ1_InternVL3-8B_8B_country_L1_prompt2.json


...

  ✔ prompt1: infer_img() full-res ok
 Saved results to r1/InternVL3-8B/RQ1_InternVL3-8B_8B_country_L1_prompt1.json


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


  ✔ prompt2: infer_img() full-res ok
 Saved results to r1/InternVL3-8B/RQ1_InternVL3-8B_8B_country_L1_prompt2.json


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


  ✔ prompt3: infer_img() full-res ok
 Saved results to r1/InternVL3-8B/RQ1_InternVL3-8B_8B_country_L1_prompt3.json
  ✔ prompt4: infer_img() full-res ok
 Saved results to r1/InternVL3-8B/RQ1_InternVL3-8B_8B_country_L1_prompt4.json
------------------------------------------------------
✅ Done.


### Download the zip file

In [ ]:
archive = "/content/results.tar.gz"
src_dir = "/content/drive/MyDrive/RQ1_Experiments/results"

!tar -czf "{archive}" -C "$(dirname "{src_dir}")" "$(basename "{src_dir}")"

from google.colab import files
files.download(archive)


tar: results/InternVL3-8B/RQ1_InternVL3-8B_8B_continent_prompt3_r1.json: file changed as we read it
tar: results/InternVL3-8B/RQ1_InternVL3-8B_8B_continent_prompt2_r1.json: file changed as we read it
tar: results/InternVL3-8B/RQ1_InternVL3-8B_8B_continent_prompt1_r1.json: file changed as we read it
tar: results/InternVL3-8B/RQ1_InternVL3-8B_8B_continent_prompt4_r1.json: file changed as we read it


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>